# Chargement et stockage des données

**Le type de données objet peut contenir plusieurs types. Par exemple, la colonne peut inclure des entiers, des réels et des chaînes de caractére qui sont collectivement étiquetés comme un objet**

In [ ]:
# importer la librairie
import pandas as pd

In [ ]:
#telecharger la fonction word_tokenize de la Bibliothèques nltk.tokenize, qui permet de couper les messages en unité linguistique individuelle donc en mot.
from nltk.tokenize import word_tokenize

In [ ]:
# charger les données du fichier sms spam.
sms = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding='latin-1')
sms.head()

# Renommer les colonnes 1 et 2 en Label et message

In [ ]:
# grâce à rename on renomme les différentes colonnes
sms= sms.rename(columns={"v1":"label","v2":"message"})
sms

# Retirer les colonnes 3, 4 et 5

In [ ]:
# grâce à dropna, on retire les 3 colonnes qui sont sans informations donc inutiles
sms = sms.dropna(axis="columns")
sms

# Afficher le diagramme de la colonne Label

In [ ]:
#importer la bibliothèque pour les diagramme
import seaborn as sns

In [ ]:
#diagramme qui affiche les spams et les hams    
sns.countplot(sms.label)

In [ ]:
# afficher le nombre de ligne et de colonne de la data frame sms
sms.shape


# Nettoyage de la colonne message

In [ ]:
# passage des lettres en majuscule à des lettres en minuscule
sms['message']=sms['message'].str.lower()
sms

In [ ]:
# importation de la variable pontuation afin de faire apparaitre toutes les ponctuations
import string
string.punctuation

In [ ]:
def remove_punct(result):
    #diviser texte en deux morceaux
    result_tok=word_tokenize(result)
    l=[]
    for token in result_tok:
        
    # test si token n'est pas dans ponc
     if not token in string.punctuation:
        l.append(token)
    
    resultat=" ".join(l)
    
    return resultat

> Dans le programme précedent, nous réalisons dans un premier temps le word tokenize, qui permet de couper les messages en unités linguistiques individuelles, donc en mot.
Dans un second temps nous créons une variable l qui est vide.
Ensuite pour un mot qui fait parti de nos mots, nous vérifions si il fait parti de la pontuation ou pas. Si le token est un mot, il est implanté dans la varible L. Si c'est une ponctuation, elle ne rentre pas dans L.
La ligne resultat= "".join(l) signifie que nous reformons les phrases.

In [ ]:
# retirer les ponctuations sur la colonne message
sms['message']=sms.message.apply(remove_punct)

In [ ]:
# afficher le fichier modifié
sms

**Les stop-words sont les mots très courants dans la langue étudiée ("et", "à", "le"... en français) qui n'apportent pas de valeur informative pour la compréhension du "sens" d'un document. Ils sont très fréquents et ralentissent notre travail : nous souhaitons donc les supprimer.**

In [ ]:
#Télècharger les stopwords grace à la librairie
from nltk.corpus import stopwords
Badword=set(stopwords.words('english'))
#stocker tout dans la variable Badword


In [ ]:
# Supprimer les stop word
def remove_stopword(result):
    
    result_tok=word_tokenize(result)
    l=[]
    for token in result_tok:
    # test si token n'est pas dans ponc
     if not token in Badword:
        l.append(token)
    
    resultat=" ".join(l)
    
    
    return resultat

> Dans le programme précedent, nous allons supprimer les stopwords,c'est à dire les petits mots qui ne sont pas utiles pour la compréhension du texte.
Nous avons donc défini la fonction remove stopwords, nous allons donc dans un premier temps séparé toute les phrases en mot. Puis nous créons une variable l vide.
dans la boucle for, nous vérifions si les mots sont des badword donc des stopwords ou non. Si le mot est un badword, il n'est pas ajouté à la varible liste l. Si le mot n'est pas un badword, il est alors placé dans la variable l.
La ligne resultat=" ".join(l) nous permet ensuite de reformer les phrases

In [ ]:
# appliquer la fonction stopword sur la colonne message
sms['message']=sms.message.apply(remove_stopword)

In [ ]:
# afficher sms avec les stopword en moins.
sms

**On remplace les mots par leur forme canonique**

In [ ]:
# On importe la fonction nltk
import nltk
from nltk.stem import WordNetLemmatizer
cano=WordNetLemmatizer()
# remplace les mots au pluriel en un mot au singulier par exemple 

In [ ]:
def canonic(result):
    
    result_tok= word_tokenize(result)
    l=[]
    
    for token in result_tok : 
        l.append(cano.lemmatize(token))
    resultat= " ".join(l)
    
    
    return resultat

> Dans ce programme, nous réalisons les mêmes deux premières étapes que pour les fonction remove punct et remove stopword.
Nous avons ensuite une boucle for, celle-ci permet de dire que lorsqu'un mot est dans nos messages, il faut réaliser le lemmatizer, donc la transformation en sa forme canonique.

In [ ]:
# Appliquer la fonction canonic sur la colonne message
sms.message=sms.message.apply(canonic)

In [ ]:
# Afficher le sms modifié 
sms

# Vectoriser la colonne des messages avec la technique de Bag of words

In [ ]:
# définir la base de donnée
corpus=sms['message'].values
corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#count car elle fait le comptage de tous les mots 
bw_vect = CountVectorizer()

# tokenize et construire le vocabulaire
#.fit veut dire je vais l'apppliquer sur
bw_fit=bw_vect.fit(corpus)

# vectoriser les mots,donc construire le tableau
bw_corpus = bw_fit.transform(corpus)

In [ ]:
bw_corpus.shape
# voir sa dimension grace à .shape

In [ ]:
# bw_fit.get_feature_names()
# .get_feature_names affiche tous les mots différents ici nous l'avons mis avec un # car la liste est très longue

In [ ]:
bw_corpus.toarray()
# afficher les valeurs, le 1 veut dire que la phrase contient une fois le mot, il peut également y avoir des 2,3,4....

In [ ]:
cv_data=pd.DataFrame(bw_corpus.toarray(),columns=bw_fit.get_feature_names())
cv_data
# Assigner dans les colonnes les mots et afficher dans les lignes les valeurs matricielles

# Vectoriser la colonne des messages avec la technique de TF-IDF


**C'est une mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document, relativement à une collection ou un corpus. Le poids augmente proportionnellement au nombre d'occurrences du mot dans le document. Il varie également en fonction de la fréquence du mot dans le corpus.**

In [ ]:
#telecharger la fonction TfidfVectorizer de la Bibliothèques sklearn.feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#Initialiser les paramètres du vectoriseur
# 10000 veut dire que nous avons laissé la visibilité sur 10000 mots. Ici ca n'a pas 
# d'importance puisque nous avons moins de mots, dans le cas ou nous aurions eu plus de mots, ca aurait réduit la quantité.
tf_vect = TfidfVectorizer(max_features=10000)
#Apprendre le vocabulaire du vectoriseur basé sur le paramètre initialisé
tfidf_fit=tf_vect.fit(corpus)
#Vectoriser le corpus
tfidf_corpus= tfidf_fit.transform(corpus)

In [ ]:
#tfidf_fit.get_feature_names()
#afficher les valeurs, ici avec un # car trop long

In [ ]:
# Afficher le tableau 
tfidf_data=pd.DataFrame(tfidf_corpus.toarray(),columns=tfidf_fit.get_feature_names())
tfidf_data

# Préparation des données pour l'algorithme apprentissage

In [ ]:
# créer deux listes une d'entrainement et une de test
from sklearn.model_selection import train_test_split
# La fonction train_test_split permet de décomposer le jeu de données en 2 groupes: les données pour l'apprentissage et les données pour les tests. Le paramètre test_size indique la taille du jeu du teste
X = cv_data
# X contient le tableau avec les chiffres de bag of words
Y = sms.label
# Y contient les infos spam ou ham
# Split train / test data :
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)
# X_train et Y_Train son pour l'apprentissage et l'arbre de décision, X_Text et Y_test sont les vraie valeuer et permettront la comparaison des résultats

# Construction de l'arbre de decision

**Un arbre de décision est un arbre orienté dont les noeuds sont étiquetés par un test et les arcs contiennent les résultats du test. On choisit de faire un test sur la variable qui disperse le mieux les classes.**

In [ ]:
#importe les données pour l'arbre
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()
#tree_model = tree.DecisionTreeClassifier(max_depth = 2)
# apprendre au logiciel qu'on utilise les deux classes pour construire l'arbre
tree_model=tree_model.fit(X_train, Y_train)

In [ ]:
# Programme pour les dimensions de l'arbre
import matplotlib.pyplot as plt
plt.figure(figsize=(15,10))
names = ['Spam', 'Ham']
tree.plot_tree(tree_model,feature_names = X.columns, 
               class_names=names,
               filled = True)

In [ ]:
# défini la variable de prédiction qui en fonction des X text nous permet de voir si on peut prédir ou non en fonction de l'arbre
Y_predict=tree_model.predict(X_test)

# Evaluation de l'arbre de décision

In [ ]:
# Plot the Confusion Matrix :
from sklearn.metrics import accuracy_score, confusion_matrix 
mat = confusion_matrix(Y_predict, Y_test)
print(mat)

In [ ]:
#plt.figure(figsize=(15,5))
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

In [ ]:
# Accuracy: c'est le pourcentage de bonne réponse
a_CART = accuracy_score(Y_test,Y_predict)
print("L'accuracy score du modèle CART est de : ",a_CART)

# Conclusion

Le tableau de prédiction à une marge d'erreur.
En conclusion, nous avons pu réaliser tout au long de nos tp des codes afin de simplifier des recherches ou de réaliser des présentation de réponses. 
Nous avons découvert le traitement naturel du langage, aussi appelé Natural Language Processing ou NLP en anglais, qui est une technologie permettant aux machines de comprendre le langage humain grâce à l’intelligence artificielle.
Nous avons ensuite pu trier des variables, grâce à des arbres de décisions ou encore des fôret d'arbre de décision. 

#  La méthode Gridsearch pour déterminer la meilleure profondeur entre 10 et 40 pour l'algorithme de l'arbre de décision

**Permet de déterminer n'importe quel paramètres et fait la recherche. Elle choisit le paramètre qui maximize la réponse**

In [ ]:
# On importe des données et la fonction Gridsearch, qui permet de déterminer la meilleur profondeur pour déterminer si le message est un spam ou un ham
from sklearn.model_selection import GridSearchCV
import numpy as np

In [ ]:

depths = np.arange(10, 40,5) # La profondeur maximale de l'arbre, toutes les valeurs entre 10 et 40 avec un pas de 5
num_features = np.arange(1,X.shape[1]) # organiser 

param_grid = [{'max_depth':depths}]
# variable pour trouver la bonne profondeur

In [ ]:
# détermine la meilleur profondeur en calaculant l'accuracy
grid_tree= GridSearchCV(estimator=tree.DecisionTreeClassifier(),param_grid=param_grid,scoring='accuracy',cv=10)
grid_tree.fit(X_train, Y_train)
# .fit relation entre X et Y train 
best_model_tree = grid_tree.best_estimator_
# la fonction best estimator, permet de selectionner la meilleure profondeur possible

In [ ]:
Y_grid=best_model_tree.predict(X_test)
# on prend le meilleur arbre de X_test pour le mettre dans Y_grid
# On utilise cet arbre pour prédir si c'est un spam ou ham

# Accuracy: pourcentage de bonne réponse de la machine par rapport à son apprentissage
accuracy_score(Y_test, Y_grid)

In [ ]:
# afficher le meilleur paramètre de profondeur
grid_tree.best_params_

# Réaliser une fôret d'arbre pour Y-train et X-train

Si on a un nombre important de variables explicatives (features). On utilise la Forêt d'arbres qui fonctionne comme le suivant:
* on prend des sous ensembles de données et des sous ensembles de variables explicatives.
* on applique l'Arbre de décision sur chaque sous ensemble.
* la prédiction de la forêt aléatoire est alors un simple vote majoritaire des arbes construites.

In [ ]:
# importer la bibliothèque pour la fôret
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Réalise une fôret d'arbre pour avoir un plus grand nombre de réponse
Rf_model = RandomForestClassifier()
Rf_model=Rf_model.fit(X_train, Y_train)

In [ ]:
Y_predict=Rf_model.predict(X_test)

In [ ]:
# prend la majorité des arbres 
mat = confusion_matrix(Y_predict, Y_test)
sns.heatmap(mat, annot=True,  xticklabels=names, yticklabels=names)
plt.xlabel('Test')
plt.ylabel('Predicted')

# Prendre un texte et dire si il est Spam ou Ham

**Le type de données objet peut contenir plusieurs types. Par exemple, la colonne peut inclure des entiers, des réels et des chaînes de caractére qui sont collectivement étiquetés comme un objet### Chatbot**

In [ ]:
text = 'Free entry in 2 a wkly comp to win FA Cup'

In [ ]:
def classer(text):

    text=remove_punct(text)
    text=remove_stopword(text)
    text=canonic(text)
    tfidf_text=tfidf_fit.transform([text])
    
    resultat = Rf_model.predict(tfidf_text)
    
    return resultat

In [ ]:
text = classer(text)

# Vérification du code

In [ ]:
a=str(input())

In [ ]:
while True:
    text = str(input("Input: "))
    if text== "exit":
        print("Response: Exiting.....")
        break
    print("Response:",classer(text))